In [25]:
import numpy as np
import pandas as pd
import json, ast

In [26]:
pwd

'/Users/mingyuanma/Desktop/HAI/util'

In [27]:
from tqdm import tqdm

In [28]:
class allaction():
    
    def __init__(self, file):
        self.df = pd.read_csv(file) 
        self.orderId, self.taskId, self.taskTicks, self.workerId, self.workerTicks = [], [], [], [], []
        self.ResponseId, self.Round, self.Tick = [], [], []
    #taskId: 0~idle, 1~chop, 2~cook, 3~plate
    #workerId: 0~idle, 1~chef, 2~sou-chef, 3~server
    #           0~idle, 1~sou-chef, 2~server
    def parse(self):
        # all id + 1, leaving 0 to be idle
        action = self.df
        for i in tqdm(np.arange(action.shape[0])):
            row = action.iloc[i,:]
            for act in ["action1", "action2", "action3"]:
                # when there is no action in this act
                if type(row[act]) != str:
                    self.orderId.append(0)
                    self.taskId.append(0)
                    self.taskTicks.append(0)
                    self.workerId.append(0)
                    self.workerTicks.append(0)
                else:
                    a = ast.literal_eval(row[act])
                    self.orderId.append(a["order_id"] + 1)
                    self.taskId.append(a["task_id"] + 1)
                    self.taskTicks.append(a["task_ticks"])
                    self.workerId.append(a["worker_id"] + 1)
                    self.workerTicks.append(a["worker_ticks"])

                self.ResponseId.append(row["ResponseId"])
                self.Round.append(row["round"])
                self.Tick.append(row["tick"])
                
    def get(self):
        self.parse()
        d = {
            "ResponseId": self.ResponseId,
            "round": self.Round,
            "tick": self.Tick,
            "orderId": self.orderId, 
            "taskId": self.taskId, 
            "taskTicks": self.taskTicks, 
            "workerId": self.workerId, 
            "workerTicks": self.workerTicks
        }
        return pd.DataFrame(d)



In [74]:
# type(pd.read_csv(file2)["action1"][1]) == float

In [73]:

# for i in np.arange(len(pd.read_csv(file2)["action1"])):
#     if type(pd.read_csv(file2)["action1"][i]) != float:
#         print(ast.literal_eval(pd.read_csv(file2)["action1"][i])["task_name"]) 

In [29]:
file1, file2 = "../data/phase1/phase1_action.csv", "../data/phase2/phase2_action.csv"

In [30]:
# action1, action2 = allaction(file1).get(), allaction(file2).get()
action2 =  allaction(file2).get()

100%|██████████| 195642/195642 [00:35<00:00, 5458.81it/s]


In [31]:
# action1

In [32]:
action2

,ResponseId,round,tick,orderId,taskId,taskTicks,workerId,workerTicks
0,R_09f8RCjNyCadwPv,1,1,1,1,2,1,0
1,R_09f8RCjNyCadwPv,1,1,2,1,2,2,0
2,R_09f8RCjNyCadwPv,1,1,3,1,2,3,0
3,R_09f8RCjNyCadwPv,1,2,0,0,0,0,0
4,R_09f8RCjNyCadwPv,1,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...
586921,R_zffMypmlgQF5Yl3,6,37,0,0,0,0,0
586922,R_zffMypmlgQF5Yl3,6,37,0,0,0,0,0
586923,R_zffMypmlgQF5Yl3,6,38,4,3,2,2,0
586924,R_zffMypmlgQF5Yl3,6,38,0,0,0,0,0


In [33]:
label = pd.read_csv("../data/phase2/phase2_control_label.csv")

In [34]:
phase2 = action2.merge(label, how="left", on="ResponseId")
phase2 = phase2.dropna()
phase2["best_tip"] = phase2["best_tip"].astype(int)
phase2

,ResponseId,round,tick,orderId,taskId,taskTicks,workerId,workerTicks,best_tip
576,R_0DNcJqALC3UG9Db,1,1,1,1,2,1,0,2
577,R_0DNcJqALC3UG9Db,1,1,2,1,2,2,0,2
578,R_0DNcJqALC3UG9Db,1,1,3,1,2,3,0,2
579,R_0DNcJqALC3UG9Db,1,2,0,0,0,0,0,2
580,R_0DNcJqALC3UG9Db,1,2,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...
585091,R_z6DSJfELElPHABP,6,34,0,0,0,0,0,3
585092,R_z6DSJfELElPHABP,6,34,0,0,0,0,0,3
585093,R_z6DSJfELElPHABP,6,35,4,3,2,2,0,3
585094,R_z6DSJfELElPHABP,6,35,0,0,0,0,0,3


In [35]:
phase2.to_csv("combined_phase2.csv", index=False)